In [2]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
import pandas as pd
import io

In [12]:
d_tr = pd.read_csv("train.csv")
d_tr = d_tr.drop(["Name","Ticket","PassengerId","Cabin"],axis=1)
d_tr

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,NaN,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


In [8]:
def feature_normalize(dataset):
    mean_data = np.mean(dataset,axis=0)
    sigma = np.std(dataset,axis=0)
    return (dataset - mean_data)/sigma

def str_to_int(df):
    str_columns = df.select_dtypes(['object']).columns
    print(str_columns)
    for col in str_columns:
        df[col] = df[col].astype('category')

    cat_columns = df.select_dtypes(['category']).columns
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
    return df

def count_space_except_nan(x):
    if isinstance(x,str):
        return x.count(" ") + 1
    else :
        return 0

def one_hot(data, cols):
    for i in cols:
        temp = pd.get_dummies(data[i], prefix=each, drop_first=False)
        del data[i]
        data = data.concat([data, temp], axis=1)
    return data
def Na_man(data,cols):
    for i in cols:
        data[i].fillna(data[i].mean(), inplace=True)
    return data


In [13]:
d_tr = pd.read_csv("train.csv")
d_tr = d_tr.drop(["Name","Ticket","PassengerId","Cabin"],axis=1)
col = ['Age','Fare']
d_tr = Na_man(d_tr,col)

['Survived' 'Pclass' 'Sex' 'Age' 'SibSp' 'Parch' 'Fare' 'Embarked']


In [14]:
d_tr

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
5,0,3,male,29.699118,0,0,8.4583,Q
6,0,1,male,54.000000,0,0,51.8625,S
7,0,3,male,2.000000,3,1,21.0750,S
8,1,3,female,27.000000,0,2,11.1333,S
9,1,2,female,14.000000,1,0,30.0708,C
